In [ ]:
import pandas as pd
import numpy as np

# Ladda data
df = pd.read_csv('../data/row_smart_home_data.csv')
print(f"Original: {len(df)} rader")
df.head()

In [ ]:
# STEG 1: Kolla problem
print("Saknade värden:", df.isnull().sum().sum())
print("Dubbletter:", df.duplicated().sum())
print("Negativ energi:", (df['Energy Consumption (kWh)'] < 0).sum())
print("Extrem energi (>20):", (df['Energy Consumption (kWh)'] > 20).sum())

In [ ]:
# STEG 2: Rensa allt på en gång
df_clean = df.copy()

# Ta bort saknade energivärden
df_clean = df_clean.dropna(subset=['Energy Consumption (kWh)'])

# Ta bort dubbletter
df_clean = df_clean.drop_duplicates()

# Ta bort felaktiga värden
df_clean = df_clean[
    (df_clean['Energy Consumption (kWh)'] >= 0) &
    (df_clean['Energy Consumption (kWh)'] <= 15) &
    (df_clean['Outdoor Temperature (°C)'] >= -40) &
    (df_clean['Outdoor Temperature (°C)'] <= 50)
]

# Rensa text
for col in ['Appliance Type', 'Season']:
    df_clean[col] = df_clean[col].str.strip()

print(f"Rensad: {len(df_clean)} rader")
print(f"Borttaget: {len(df) - len(df_clean)} rader")

In [ ]:
# STEG 3: Kontrollera resultat
print("EFTER RENSNING:")
print("Saknade värden:", df_clean.isnull().sum().sum())
print("Dubbletter:", df_clean.duplicated().sum())
print("Negativ energi:", (df_clean['Energy Consumption (kWh)'] < 0).sum())
print("Extrem energi:", (df_clean['Energy Consumption (kWh)'] > 15).sum())

df_clean.describe()

In [ ]:
# STEG 4: Spara ren data
df_clean.to_csv('../data/clean_data.csv', index=False)
print("✅ Ren data sparad som: ../data/clean_data.csv")
print(f"📊 {len(df_clean)} rader × {len(df_clean.columns)} kolumner")

In [ ]:
# BONUS: Snabb analys av ren data
import matplotlib.pyplot as plt

# Energi per apparat
energy_by_appliance = df_clean.groupby('Appliance Type')['Energy Consumption (kWh)'].mean().round(2)
print("Genomsnittlig energi per apparat:")
print(energy_by_appliance)

# Enkel plot
energy_by_appliance.plot(kind='bar', figsize=(10, 5), color='lightblue')
plt.title('Energiförbrukning per Apparat (Rensad Data)')
plt.ylabel('kWh')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()